In [7]:
# 17_search.ipynb

%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/app')

from IPython.display import Markdown

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
from hybrid_search import hybrid_search

In [9]:
from qdrant_search import QdrantSearchEngine
from bm25_search import ElasticsearchBM25
voyage_apikey = 'pa-g4x1yHL4Hv6qBjXcrT2Nhm5gG0srlDhVBp6Op7_IJBQ'


qdrant_engine = QdrantSearchEngine(voyage_apikey)
es_engine = await ElasticsearchBM25.create() 

Connecting to Elasticsearch at elasticsearch:9200


In [10]:
q = """
meeting notetakers
"""

In [11]:
import os
QDRANT_HOST = os.getenv('QDRANT_HOST')
QDRANT_PORT = os.getenv('QDRANT_PORT')

In [12]:
from qdrant_client import AsyncQdrantClient

In [13]:
query = "software"
content_ids = None
k = 100

In [8]:
semantic_results = []
if qdrant_engine:
    semantic_results = await qdrant_engine.search(
        query=query,
        content_ids=content_ids,
        k=k
    )
text_results = []
if es_engine:
    text_results = await es_engine.search(
        query=query,
        content_ids=content_ids,
        k=k
    )

/usr/local/lib/python3.11/site-packages/voyageai/client.py:58: UserWarning: The `model` argument is not specified and defaults to voyage-2. It will be a required argument in the future. We recommend to specify the model when using this function. Please see https://docs.voyageai.com/docs/embeddings for the list of latest models provided by Voyage AI.
  warnings.warn(


In [ ]:
len(semantic_results)

92

In [ ]:
len(text_results)

4

In [11]:
# Merge results
all_results = []
seen_content = set()

# Process semantic results first
for result in semantic_results:
    content_id = result.payload.get('content_id')
    if content_id not in seen_content:
        seen_content.add(content_id)
        all_results.append({
            'score': result.score,
            'content': result.payload.get('content', ''),
            'content_id': content_id,
            'timestamp': result.payload.get('timestamp'),
            'formatted_time': result.payload.get('formatted_time'),
            'contextualized_content': result.payload.get('contextualized_content', ''),
            'content_type': result.payload.get('content_type', '')
            
            
            
            
        })

In [35]:
semantic_results[0].payload

{'content_id': 'fb996504-0475-4ee9-a85e-dda4e9f15a4b',
 'timestamp': '2025-01-15T14:07:51.364000+00:00',
 'formatted_time': '2025-01-15 14:07:51',
 'content': 'Alex Shevliakov:  Юзер приходит посмотреть звонки.  находит какой-то звонок, он нажимает на...  Что он делает с этим звонком?  Он нажимает на этот звонок, смотрит инфу или чатится по этому звонку, спрашивает что-то?  Ну, в смысле, он смотрит транскрипт summary или чатится и спрашивает по этому звонку инфу?  Смотрит инфу. Значит, она там должна быть либо подготовлена, либо должна на лету готовиться, если смотрит инфу.  А что такое инфа? Я нажимаю и смотрю.  Значит, мне готовится какой-то summary на лету или из кэша подгружается, если оно.  Слушай, мне кажется, как основа, наверное, если в базе уже есть транскрипт, то, наверное, он есть, то можно показывать транскрипт, а если он...  уже запросил нет если он запросил самаре то можно ему сделать самый тип он наш ...транскрипт, там будет кнопочка Generate Summary.  Ты знаешь, что мне

In [12]:
len(all_results)

4

In [13]:

# Process text results
for hit in text_results['hits']['hits']:
    content_id = hit['_source'].get('content_id')
    if content_id not in seen_content:
        seen_content.add(content_id)
        all_results.append({
            'score': hit['_score'],
            'content': hit['_source'].get('content', ''),
            'content_id': content_id,
            'timestamp': hit['_source'].get('timestamp'),
            'formatted_time': hit['_source'].get('formatted_time'),
            'contextualized_content': hit['_source'].get('contextualized_content', ''),
            'content_type': hit['_source'].get('content_type', '')
        })

# Sort by score descending
all_results.sort(key=lambda x: x['score'], reverse=True)

In [14]:
all_results

[{'score': 0.047420308,
  'content': 'Alex Shevliakov:  Юзер приходит посмотреть звонки.  находит какой-то звонок, он нажимает на...  Что он делает с этим звонком?  Он нажимает на этот звонок, смотрит инфу или чатится по этому звонку, спрашивает что-то?  Ну, в смысле, он смотрит транскрипт summary или чатится и спрашивает по этому звонку инфу?  Смотрит инфу. Значит, она там должна быть либо подготовлена, либо должна на лету готовиться, если смотрит инфу.  А что такое инфа? Я нажимаю и смотрю.  Значит, мне готовится какой-то summary на лету или из кэша подгружается, если оно.  Слушай, мне кажется, как основа, наверное, если в базе уже есть транскрипт, то, наверное, он есть, то можно показывать транскрипт, а если он...  уже запросил нет если он запросил самаре то можно ему сделать самый тип он наш ...транскрипт, там будет кнопочка Generate Summary.  Ты знаешь, что мне генерить для каждого созвона summary лишний раз.  Если оно не нужно, не показывайте это summary.  Может быть так. Хорошо.

In [17]:
query = "vexa"

In [18]:
r = await hybrid_search(query, qdrant_engine, es_engine)

/usr/local/lib/python3.11/site-packages/voyageai/client.py:58: UserWarning: The `model` argument is not specified and defaults to voyage-2. It will be a required argument in the future. We recommend to specify the model when using this function. Please see https://docs.voyageai.com/docs/embeddings for the list of latest models provided by Voyage AI.
  warnings.warn(


In [19]:
import pandas as pd
df = pd.DataFrame(r['results'])

In [20]:
df

,score,content,content_id,timestamp,formatted_time,contextualized_content,content_type,topic,speaker,speakers
0,0.044242,Alex Shevliakov: Юзер приходит посмотреть зво...,fb996504-0475-4ee9-a85e-dda4e9f15a4b,2025-01-15T14:07:51.364000+00:00,2025-01-15 14:07:51,The chunk is part of a discussion between Alex...,meeting,User Actions on Calls,Alex Shevliakov,"[Alex Shevliakov, Dmitriy Grankin]"
1,0.040366,Artem Puzik: это же внутри уже готового прило...,580bb613-00de-4c4a-b048-2c2576fcdef0,2025-01-23T09:40:30.720000+00:00,2025-01-23 09:40:30,This chunk is part of a conversation between A...,meeting,User Authentication,Artem Puzik,"[Artem Puzik, Sergey Ryabenko, Dmitry Grankin]"
2,0.029616,Dmitriy Grankin: чуть попозже сделаем вот в ч...,6a7287fb-b6ae-46cf-8b99-1ce1bcc81403,2025-01-21T15:36:56.606000+00:00,2025-01-21 15:36:56,The chunk is part of a discussion among Dmitri...,meeting,User Flow Discussion,Dmitriy Grankin,"[Artem Puzik, Dmitriy Grankin, Sergey Ryabenko]"
3,0.003163,Mariana Montenegro: half an hour yeah okay um...,a0c3f703-2a5e-4ed0-9561-807121516984,2025-01-21T15:02:50.184000+00:00,2025-01-21 15:02:50,This chunk is part of a conversation where Mar...,meeting,Introduction to AI Hub,Mariana Montenegro,"[Mariana Montenegro, Dmitry Grankin, Pedro Tei..."


In [24]:
def format_meeting_context(df, show_content=True, show_contextualized=True):
    unique_meetings = df['content_id'].unique()
    meeting_map = {mid: idx+1 for idx, mid in enumerate(unique_meetings)}
    meeting_map_reverse = {v:k for k,v in meeting_map.items()}
    
    # Sort by timestamp for meetings, then by formatted_time within meetings
    df = df.sort_values(['timestamp', 'formatted_time'])
    
    meetings = []
    for content_id, group in df.groupby('content_id'):
        int_content_id = meeting_map[content_id]
        timestamp = pd.to_datetime(group['timestamp'].iloc[0])
        # Get content type from first row of group, capitalize first letter
        content_type = group['content_type'].iloc[0].capitalize()
        date_header = f"## {content_type} {int_content_id} - {timestamp.strftime('%B %d, %Y %H:%M')}"
        
        content_items = []
        for _, row in group.iterrows():
            time_prefix = f"[{row['formatted_time']}]" if row['formatted_time'] else ''
            if show_contextualized:
                content_items.append(f"- {time_prefix} {row['contextualized_content']}")
            if show_content:
                prefix = "  > " if show_contextualized else "- "
                content_items.append(f"{prefix}{row['content']}")
        
        content = "\n".join(content_items)
        meetings.append(f"{date_header}\n\n{content}\n")
    
    return "\n".join(meetings), meeting_map_reverse

In [32]:
    
# Create content maps
unique_contents = df['content_id'].unique()
content_map = {cid: idx+1 for idx, cid in enumerate(unique_contents)}
content_map_reverse = {v: k for k, v in content_map.items()}

contents = []
for content_id, group in df.groupby('content_id'):
    int_content_id = content_map[content_id]
    timestamp = pd.to_datetime(group['timestamp'].iloc[0])
    date_header = f"##  [{int_content_id}] - {timestamp.strftime('%B %d, %Y %H:%M')}"
    
    content_items = []
    for _, row in group.iterrows():
        time_prefix = f"[{row['formatted_time']}]" if row['formatted_time'] else ''
        if 'contextualized_content' in row:
            content_items.append(f"- {time_prefix} {row['contextualized_content']}")
        content_items.append(f"  > {row['content']}")
    
    content = "\n".join(content_items)
    contents.append(f"{date_header}\n\n{content}\n")

In [34]:
display(Markdown("\n".join(contents)))

##  [2] - January 23, 2025 09:40

- [2025-01-23 09:40:30] This chunk is part of a conversation between Artem Puzik and Dmitry Grankin discussing the integration of an API within an existing application. Artem is clarifying the authentication process and the handling of user tokens within the app, emphasizing that the user will already be authorized when accessing the chat feature.
  > Artem Puzik:  это же внутри уже готового приложения будет правильно ну то есть пользователь туда уже на эту страницу он придет авторизована или это просто что-то отдельная мне как бы окошко там авторизации я да если ты сейчас пойдешь там у тебя есть еще  можно взять пока ту страницу, которая есть, она там неадаптивная, надо будет адаптивнее справиться, но это потом.  там слово такое что сначала сначала у тебя идет google авторизация и сначала как пользователь  Нет, Дим, я о другом.  То есть, допустим, если моя страница, мой компонент или что-то там, API мое будет лежать уже внутри страницы, Твоего приложения, то есть авторизация, значит, уже пройдена.  Значит, этот токен у тебя где-то там лежит, я его найду, правильно?  Этот токен, ты его получаешь от приложения.  Женя, у тебя там как раз описано, как авторизация происходит, и ты его сохраняешь в руке.  Еще раз.  Да. Для того, чтобы дойти до этого чата, я уже у тебя в существующем Vexo App, да, уже какой-то путь прошел, правильно?  Авторизация.

##  [3] - January 21, 2025 15:36

- [2025-01-21 15:36:56] The chunk is part of a discussion among Dmitriy Grankin, Artem Puzik, and Sergey Ryabenko about the user flow and functionality of an application that includes chat and meeting features. In this specific segment, Dmitriy outlines the first user flow related to accessing and interacting with meeting information, including the use of a meeting card and context for chat interactions.
  > Dmitriy Grankin:  чуть попозже сделаем вот в чем здесь ну как бы первый юзер флоу до начнем под давайте по user flow пойдем первый это Я хочу что-нибудь посмотреть по встрече, которая только прошла или какая-нибудь другая.  Я нажимаю на карт, на митинг-карт, у меня открывается...  Правая панель, в которой у меня есть транскрипция. И чат работает сейчас в контексте этой встречи. Контекст, да, это то, что мы подаем.  скажем так до чата да то есть контекст имеет разным образом определяется самый простой флоу от когда мы выбрали встречу, и он определяется по этой встрече, соответственно, по этой встрече.  Можем что-нибудь спросить про эту встречу, можем выбрать Quick Action.  Quick Action – это такой промпт, который исполняется автоматически при наступлении.

##  [4] - January 21, 2025 15:02

- [2025-01-21 15:02:50] This chunk is part of a conversation where Mariana Montenegro introduces herself and the AI hub at Unicorn Factory Lisboa, discussing its purpose and community focus, as part of a larger dialogue involving Dmitry Grankin and Pedro Teixeira about AI startups and collaboration opportunities.
  > Mariana Montenegro:  half an hour yeah okay um so i think pedro is running a bit late but but well i will introduce myself firstly so my name is mariana i'm the manager the responsible for the ai hub on parts of the unicorn factory lisboa So, the AI hub is more than building with offices.

##  [1] - January 15, 2025 14:07

- [2025-01-15 14:07:51] The chunk is part of a discussion between Alex Shevliakov and Dmitriy Grankin about the user experience and functionality of a platform that involves calls and chat interactions. They are specifically focusing on how users access and interact with call information, including viewing transcripts and generating summaries, as well as the conditions under which this information is displayed.
  > Alex Shevliakov:  Юзер приходит посмотреть звонки.  находит какой-то звонок, он нажимает на...  Что он делает с этим звонком?  Он нажимает на этот звонок, смотрит инфу или чатится по этому звонку, спрашивает что-то?  Ну, в смысле, он смотрит транскрипт summary или чатится и спрашивает по этому звонку инфу?  Смотрит инфу. Значит, она там должна быть либо подготовлена, либо должна на лету готовиться, если смотрит инфу.  А что такое инфа? Я нажимаю и смотрю.  Значит, мне готовится какой-то summary на лету или из кэша подгружается, если оно.  Слушай, мне кажется, как основа, наверное, если в базе уже есть транскрипт, то, наверное, он есть, то можно показывать транскрипт, а если он...  уже запросил нет если он запросил самаре то можно ему сделать самый тип он наш ...транскрипт, там будет кнопочка Generate Summary.  Ты знаешь, что мне генерить для каждого созвона summary лишний раз.  Если оно не нужно, не показывайте это summary.  Может быть так. Хорошо.


In [25]:
input_context, meeting_id_map = format_meeting_context(df,show_contextualized=True)

In [26]:
display(Markdown(input_context))

## Meeting 2 - January 23, 2025 09:40

- [2025-01-23 09:40:30] This chunk is part of a conversation between Artem Puzik and Dmitry Grankin discussing the integration of an API within an existing application. Artem is clarifying the authentication process and the handling of user tokens within the app, emphasizing that the user will already be authorized when accessing the chat feature.
  > Artem Puzik:  это же внутри уже готового приложения будет правильно ну то есть пользователь туда уже на эту страницу он придет авторизована или это просто что-то отдельная мне как бы окошко там авторизации я да если ты сейчас пойдешь там у тебя есть еще  можно взять пока ту страницу, которая есть, она там неадаптивная, надо будет адаптивнее справиться, но это потом.  там слово такое что сначала сначала у тебя идет google авторизация и сначала как пользователь  Нет, Дим, я о другом.  То есть, допустим, если моя страница, мой компонент или что-то там, API мое будет лежать уже внутри страницы, Твоего приложения, то есть авторизация, значит, уже пройдена.  Значит, этот токен у тебя где-то там лежит, я его найду, правильно?  Этот токен, ты его получаешь от приложения.  Женя, у тебя там как раз описано, как авторизация происходит, и ты его сохраняешь в руке.  Еще раз.  Да. Для того, чтобы дойти до этого чата, я уже у тебя в существующем Vexo App, да, уже какой-то путь прошел, правильно?  Авторизация.

## Meeting 3 - January 21, 2025 15:36

- [2025-01-21 15:36:56] The chunk is part of a discussion among Dmitriy Grankin, Artem Puzik, and Sergey Ryabenko about the user flow and functionality of an application that includes chat and meeting features. In this specific segment, Dmitriy outlines the first user flow related to accessing and interacting with meeting information, including the use of a meeting card and context for chat interactions.
  > Dmitriy Grankin:  чуть попозже сделаем вот в чем здесь ну как бы первый юзер флоу до начнем под давайте по user flow пойдем первый это Я хочу что-нибудь посмотреть по встрече, которая только прошла или какая-нибудь другая.  Я нажимаю на карт, на митинг-карт, у меня открывается...  Правая панель, в которой у меня есть транскрипция. И чат работает сейчас в контексте этой встречи. Контекст, да, это то, что мы подаем.  скажем так до чата да то есть контекст имеет разным образом определяется самый простой флоу от когда мы выбрали встречу, и он определяется по этой встрече, соответственно, по этой встрече.  Можем что-нибудь спросить про эту встречу, можем выбрать Quick Action.  Quick Action – это такой промпт, который исполняется автоматически при наступлении.

## Meeting 4 - January 21, 2025 15:02

- [2025-01-21 15:02:50] This chunk is part of a conversation where Mariana Montenegro introduces herself and the AI hub at Unicorn Factory Lisboa, discussing its purpose and community focus, as part of a larger dialogue involving Dmitry Grankin and Pedro Teixeira about AI startups and collaboration opportunities.
  > Mariana Montenegro:  half an hour yeah okay um so i think pedro is running a bit late but but well i will introduce myself firstly so my name is mariana i'm the manager the responsible for the ai hub on parts of the unicorn factory lisboa So, the AI hub is more than building with offices.

## Meeting 1 - January 15, 2025 14:07

- [2025-01-15 14:07:51] The chunk is part of a discussion between Alex Shevliakov and Dmitriy Grankin about the user experience and functionality of a platform that involves calls and chat interactions. They are specifically focusing on how users access and interact with call information, including viewing transcripts and generating summaries, as well as the conditions under which this information is displayed.
  > Alex Shevliakov:  Юзер приходит посмотреть звонки.  находит какой-то звонок, он нажимает на...  Что он делает с этим звонком?  Он нажимает на этот звонок, смотрит инфу или чатится по этому звонку, спрашивает что-то?  Ну, в смысле, он смотрит транскрипт summary или чатится и спрашивает по этому звонку инфу?  Смотрит инфу. Значит, она там должна быть либо подготовлена, либо должна на лету готовиться, если смотрит инфу.  А что такое инфа? Я нажимаю и смотрю.  Значит, мне готовится какой-то summary на лету или из кэша подгружается, если оно.  Слушай, мне кажется, как основа, наверное, если в базе уже есть транскрипт, то, наверное, он есть, то можно показывать транскрипт, а если он...  уже запросил нет если он запросил самаре то можно ему сделать самый тип он наш ...транскрипт, там будет кнопочка Generate Summary.  Ты знаешь, что мне генерить для каждого созвона summary лишний раз.  Если оно не нужно, не показывайте это summary.  Может быть так. Хорошо.


In [27]:
[v for v in meeting_id_map.values()]

['fb996504-0475-4ee9-a85e-dda4e9f15a4b',
 '580bb613-00de-4c4a-b048-2c2576fcdef0',
 '6a7287fb-b6ae-46cf-8b99-1ce1bcc81403',
 'a0c3f703-2a5e-4ed0-9561-807121516984']

In [28]:
from core import generic_call,system_msg,user_msg,count_tokens

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fe9ea2d6450>


In [29]:
input = count_tokens(input_context)

In [30]:
input

1199

In [25]:
from core import generic_call,system_msg,user_msg


In [26]:
system_prompt = """
you are helpful assistant that is answering user requests by interpreting  while interpreting the context
You have to rely on the context to answer the user request

User is Dmitry Grankin, Address his as YOU and distinguish them in in the provided context in in the response.

Always supply reference to specific meeting with [#meeting_id]

If appropriate, inlude qoutes with speaker reference

whenever you're responding consider everything you know about me in the memory to form a context of things that I would find interesting and where possible link back to those topics and include key terminologies and concepts that will help expand my knowledge along those areas.
"""




In [27]:

r = await generic_call([system_msg(system_prompt),
                        user_msg(f'context: {input_context}'),
                        user_msg(f'request: {q}')])
r



To establish a system for meeting notetakers, you might consider the following approach based on your recent discussions:

1. **Assign Notetakers**: Designate a specific person to take notes for each meeting. This could rotate among team members to ensure everyone is involved. For instance, in your recent meetings, you could have Artem Puzik or Alex Shevliakov take notes during their respective discussions.

2. **Structured Format**: Use a consistent format for notes that includes:
   - Date and time of the meeting
   - Attendees
   - Key discussion points
   - Action items with assigned responsibilities
   - Follow-up dates

3. **Documentation**: Ensure that the notes are documented in a shared space (like a project management tool or a shared document) where all team members can access them. This aligns with your discussions about API documentation and component creation, as mentioned in [#meeting_2].

4. **Review Process**: At the beginning of each meeting, briefly review the notes 

'To establish a system for meeting notetakers, you might consider the following approach based on your recent discussions:\n\n1. **Assign Notetakers**: Designate a specific person to take notes for each meeting. This could rotate among team members to ensure everyone is involved. For instance, in your recent meetings, you could have Artem Puzik or Alex Shevliakov take notes during their respective discussions.\n\n2. **Structured Format**: Use a consistent format for notes that includes:\n   - Date and time of the meeting\n   - Attendees\n   - Key discussion points\n   - Action items with assigned responsibilities\n   - Follow-up dates\n\n3. **Documentation**: Ensure that the notes are documented in a shared space (like a project management tool or a shared document) where all team members can access them. This aligns with your discussions about API documentation and component creation, as mentioned in [#meeting_2].\n\n4. **Review Process**: At the beginning of each meeting, briefly rev

In [28]:
display(Markdown(r))

To establish a system for meeting notetakers, you might consider the following approach based on your recent discussions:

1. **Assign Notetakers**: Designate a specific person to take notes for each meeting. This could rotate among team members to ensure everyone is involved. For instance, in your recent meetings, you could have Artem Puzik or Alex Shevliakov take notes during their respective discussions.

2. **Structured Format**: Use a consistent format for notes that includes:
   - Date and time of the meeting
   - Attendees
   - Key discussion points
   - Action items with assigned responsibilities
   - Follow-up dates

3. **Documentation**: Ensure that the notes are documented in a shared space (like a project management tool or a shared document) where all team members can access them. This aligns with your discussions about API documentation and component creation, as mentioned in [#meeting_2].

4. **Review Process**: At the beginning of each meeting, briefly review the notes from the previous meeting to ensure everyone is on the same page and to follow up on action items.

5. **Feedback Loop**: Encourage team members to provide feedback on the notes to improve clarity and completeness over time.

By implementing these strategies, you can enhance collaboration and ensure that important information is captured and easily accessible for future reference. This will also support your ongoing work on the user interface components and API integration discussed in your meetings.

In [29]:
def extract_tagged_content(text: str, tag: str) -> str:
    import re
    # Try exact match first
    pattern = f"<{tag}>(.*?)</{tag}>"
    match = re.search(pattern, text, re.DOTALL)
    if match:
        return match.group(1).strip()
    
    # If no exact match, try to find content after opening tag
    pattern = f"<{tag}>(.*?)(?:</?|$)"
    match = re.search(pattern, text, re.DOTALL)
    return match.group(1).strip() if match else ""

# Example usage:
thinking = extract_tagged_content(r, "thinking")
answer = extract_tagged_content(r, "answer")

In [30]:
thinking

''